# Using a String Template with Gemini for Pair Programming

#### Setup
Set the Gemini API key with the provided helper function.

In [1]:
#!pip install python-dotenv
import os
from dotenv import load_dotenv, find_dotenv

def get_api_key():
    
    _ = load_dotenv(find_dotenv()) # read local .env file
    return os.getenv('GOOGLE_API_KEY')

# PaLM legacy
## import google.generativeai as palm
## palm.configure(api_key=get_api_key())

# Gemini API
# From now own with Gemini API
import os
import google.generativeai as genai
from google.api_core import client_options as client_options_lib

genai.configure(
    api_key=get_api_key(),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

#### Pick the model that generates text

In [2]:
models = [m for m in genai.list_models() if 'generateText' in m.supported_generation_methods]
model_bison = models[0]
model_bison

Model(name='models/text-bison-001',
      base_model_id='',
      version='001',
      display_name='PaLM 2 (Legacy)',
      description='A legacy model that understands text and generates text as an output',
      input_token_limit=8196,
      output_token_limit=1024,
      supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'],
      temperature=0.7,
      top_p=0.95,
      top_k=40)

In [3]:
# Set the model to connect to the Gemini API
model_flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

### Legacy PaLM API
#### Helper function to call the PaLM API
```Python
from google.api_core import retry
@retry.Retry()
def generate_text(prompt, 
                  model=model_bison, 
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)
```

### Helper function to call the Gemini API

In [4]:
def generate_text(prompt,
                  model=model_flash,
                  temperature=0.0):
    return model_flash.generate_content(prompt,
                                  generation_config={'temperature':temperature})

#### Prompt template

1. priming: getting the LLM ready for the type of task you'll ask it to do.
2. question: the specific task.
3. decorator: how to provide or format the output.

In [5]:
prompt_template = """
{priming}

{question}

{decorator}

Your solution:
"""

In [6]:
priming_text = "You are an expert at writing clear, concise, Python code."

In [7]:
question = "create a doubly linked list"

#### Observe how the decorator affects the output
- In other non-coding prompt engineering tasks, it's common to use "chain-of-thought prompting" by asking the model to work through the task "step by step".
- For certain tasks like generating code, you may want to experiment with other wording that would make sense if you were asking a developer the same question.

In the code cell below, try out option 1 first, then try out option 2.

In [8]:
# option 1
# decorator = "Work through it step by step, and show your work. One step per line."

# option 2
decorator = "Insert comments for each line of code."

In [9]:
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)

#### review the prompt

In [10]:
print(prompt)


You are an expert at writing clear, concise, Python code.

create a doubly linked list

Insert comments for each line of code.

Your solution:



#### Call the API to get the completion

In [11]:
completion = generate_text(prompt)
# Gemini API
print(completion.text)

# PaLM legacy
## print(completion.result)

```python
class Node:
    """Represents a node in the doubly linked list."""
    def __init__(self, data):
        """Initializes a node with given data."""
        self.data = data  # Store the data in the node
        self.prev = None  # Pointer to the previous node (initially None)
        self.next = None  # Pointer to the next node (initially None)


class DoublyLinkedList:
    """Represents a doubly linked list."""
    def __init__(self):
        """Initializes an empty doubly linked list."""
        self.head = None  # Head of the list (initially None)
        self.tail = None  # Tail of the list (initially None)

    def append(self, data):
        """Appends a new node with the given data to the end of the list."""
        new_node = Node(data)  # Create a new node with the data
        if not self.head:  # If the list is empty
            self.head = new_node  # Set the new node as both head and tail
            self.tail = new_node
        else:
            new_node.prev = s

#### Try another question

In [12]:
question = """create a very large list of random numbers in python, 
and then write code to sort that list"""

In [13]:
prompt = prompt_template.format(priming=priming_text,
                                question=question,
                                decorator=decorator)

In [14]:
print(prompt)


You are an expert at writing clear, concise, Python code.

create a very large list of random numbers in python, 
and then write code to sort that list

Insert comments for each line of code.

Your solution:



In [15]:
completion = generate_text(prompt)
print(completion.text)

```python
import random
import time

# Define the size of the list (adjust as needed, but be mindful of memory limitations)
list_size = 1000000  

# Generate a list of random numbers. Using random.random() for floating-point numbers.  For integers, use random.randint(a,b)
random_numbers = [random.random() for _ in range(list_size)]  

# Measure the time it takes to sort the list using Python's built-in Timsort (a hybrid sorting algorithm)
start_time = time.time()  #Record the start time

#Sort the list in place using the list.sort() method. This is generally the most efficient way to sort large lists in Python.
random_numbers.sort()  

end_time = time.time()  #Record the end time
elapsed_time = end_time - start_time  #Calculate the elapsed time

# Print the elapsed time.  The time will vary depending on your system's processing power.
print(f"Time taken to sort {list_size} numbers: {elapsed_time:.4f} seconds")


#Verification (optional): Check if the list is sorted.  Uncomment to use. 

#### Try out the generated code
- Debug it as needed.  For instance, you may need to import `random` in order to use the `random.randint()` function.

In [16]:
import random
import time

# Define the size of the list (adjust as needed, but be mindful of memory limitations)
list_size = 1000000  

# Generate a list of random numbers. Using random.random() for floating-point numbers.  For integers, use random.randint(a,b)
random_numbers = [random.random() for _ in range(list_size)]  

# Measure the time it takes to sort the list using Python's built-in Timsort (a hybrid sorting algorithm)
start_time = time.time()  #Record the start time

#Sort the list in place using the list.sort() method. This is generally the most efficient way to sort large lists in Python.
random_numbers.sort()  

end_time = time.time()  #Record the end time
elapsed_time = end_time - start_time  #Calculate the elapsed time

# Print the elapsed time.  The time will vary depending on your system's processing power.
print(f"Time taken to sort {list_size} numbers: {elapsed_time:.4f} seconds")

Time taken to sort 1000000 numbers: 0.2258 seconds
